In [ ]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: 2
### Team Member Names: Ole, Rudra, Sheraz
### Team Strategy Chosen: Risk-Free


Our strategy is Risk-Free, meaning that we want to create a stock portfolio that has returns as close to 0 as possible (final portfolio value is as close to the initial investment of ($1 million - fees) as possible). 
*The methods and types of data we used all came from Prof. Thompson's lectures 

This is our strategy: (the strategy will be explained more in the code as well)
1. Remove all unwanted stocks and months:
    - Remove all foreign stocks, keep only the US and Canadian stocks
    - Remove all stocks with an average daily volume < 5,000 shares between Oct 1, 2024 and Sep 30, 2025 
    - Drop months for each stock with < 18 trading days

2. After removing all the unwanted stocks, get data for each of the remaining stocks that will help us determine if they are stable and optimal for us to add to our portfolio:
    - Standard deviation
        - Tells us how volatile/risky the stock is, and we will look for stocks that have lower standard deviations (close to 0) to reduce the volatility and risk of our portfolio
    - Beta to the S&P 500
        - Tells us how closely the stock moves with the market. We want to look for stocks that have lower betas (close to 0 in absolute value) because it will mean that the stock's movements aren't very correlated with the market, and don't particularly go up or down when the market moves
    - Expected Monthly Returns
        - Tells us what percentage return we should expect to see per month. We want to look for stocks that have lower expected monthly returns (close to 0 in absolute value) because we want a portfolio that gives as close to 0% returns as possible

3. Now that we have data for all the stocks, combine the data into a score and rank the stocks on the 3 metrics, getting a sorted list in order of which stocks are the best to invest in

4. Based on the ranking, add stocks to the portfolio one at a time 
    - We want 25 stocks (maximum amount allowed) at the end, with each being weighed equally because we want to diversify as much as possible and not get hurt by company-based scandals or industry-wide down/upturns. 
    - First, add the large and small-cap stocks that rank the highest in the scores
    - We don't just choose the top 25 stocks based on the ranking because that ranking doesn't take into account how correlated each stock is to one another
    - For example, if the top 25 stocks are all in the same industry, then they will probably be highly correlated with each other, making our portfolio not very diversified
    - This is why we will add the stocks one at a time and making sure that each additional stock is not too correlated with the stocks already in the portfolio
    - For each of the next stocks (going through the stocks based on their rankings):
        - If there are already 10 (0.4 x 25) stocks in the portfolio that are in the same industry, then skip
        - Get the average correlation between them and each of the existing stocks already in the portfolio
        - If it’s below a certain threshold, then add it to the portfolio
        - For the first few stocks, we will make the threshold higher/more lenient since there are fewer stocks to compare it to
        - Later on, if we have already chosen a few of our stocks, we will reduce the threshold, since if the average correlation for a stock is still high, then it means that it's strongly correlated with many of the stocks already in the portfolio, which is not optimal
        - If there are already 25 stocks, then break out the loop/stop
        - Change the correlation thresholds and run through the loop again, adding one stock at a time to an empty portfolio

    - After every iteration, we will have a portfolio of stocks. (If the portfolio has fewer than 25 stocks, skip this part)
    - We will find the expected daily return of that portfolio, and if the absolute value of the return is lower than that of our current best portfolio (closer to 0), then this becomes the new best portfolio
    - After all possible correlation thresholds are gone through, we will have the most optimal portfolio

5. Purchase the 25 stocks in the portfolio
    - Give equal (4%) weighting to each stock
    - Deduct fees for purchasing, either 2.15 USD flat or 0.001 USD per share purchased (whichever is smaller)


In [178]:
#tickers_df = pd.read_csv('Tickers_Example.csv')

# Note: Assignment said that the code should read from a 'Tickers.csv' for the TAs to run the secret csv, 
# but all the code outputs are based on the 'Tickers_Example.csv' above
tickers_df = pd.read_csv('Tickers.csv')

start_date = '2024-10-01'
end_date = '2025-09-30'

# list of all the tickers in the CSV file
ticker_list = tickers_df.iloc[:, 0].tolist()


<font color='red'>
Brief note before going into the code changes:
    
In our original submission, we did account for all the requirements stated in the document, such as only US/Canadian companies, no stocks with average trade volume < 5000, sector requirements, tickers that had no data from yfinance, tickers that were repeated, etc., however the code couldn't run because of a few tricks that weren't mentioned in the document, so this resubmission fixes those:
1. Some tickers in the file were in lowercase instead of uppercase
2. Some of the tickers weren't stocks (though the assignment description did say that "Your portfolio will be run on a secret list of <font color='green'>stocks</font>"), so we were under the assumption that the tickers were all stocks
3. Some tickers had extra spaces in their name (ex. " SU.TO" instead of "SU.TO")
4. Although our original submission already dealt with duplicate tickers that had the exact same name, this new ticker file had some tickers that were the same stock, but in different markets. The assignment document doesn't say anything against having a portfolio with both SHOP.TO and SHOP, for example, but we still treated it as a duplicate in this resubmission and removed one of them just to be sure.

The rest of the red markdown boxes and code boxes directly underneath will be either for new code that fixes one of the bullet points, or extra explanation for how the code filtered tickers in the original submission
</font>

<font color='red'>NEW CODE:
Some of the tickers in the ticker list have spaces in front of them, and some of the tickers are lowercase instead of uppercase.
This code removes all unecessary spaces and changes lowercase letters to uppercase for all the tickers
</font>

In [179]:
# NEW CODE
new_ticker_list = []
for ticker in ticker_list:
    new_ticker_list.append(ticker.replace(' ', '').upper())

ticker_list = new_ticker_list.copy()
ticker_list

['ASDFA.TO',
 'SU.TO',
 'ABBV',
 'LOW',
 'AUST',
 'TD',
 'AMZN',
 'AXP',
 'GCT',
 'BK',
 'HDFC.NS',
 'SQ',
 'CELG',
 'EXE.TO',
 'CMCSA',
 'SHOP',
 'COST',
 'CSCO',
 'GM',
 'GOOG',
 'AIM.TO',
 'FTG.TO',
 'ORCL',
 'SAP.TO',
 'DUOL',
 'PEP',
 'T.TO',
 'SLB',
 'SO',
 'SPG',
 'SHOP.TO',
 'COST',
 'GC=F',
 'T.TO',
 'RY.TO',
 'TD.TO',
 'AW.TO',
 'BTC-USD',
 'VSP.TO',
 'XZO']

<font color='red'>NEW CODE:
some of the tickers are duplicated, but in different markets, such as SHOP vs. SHOP.TO, so we only keep one of them.
</font>

In [180]:
visited = []
new_ticker_list = []
for ticker in ticker_list:
    # find if the actual stock name (without the .TO or any market) already exists in the ticker list
    temp_ticker = ticker.split('.')
    temp_ticker = temp_ticker[0]
    if temp_ticker in visited:
        continue
    new_ticker_list.append(ticker)
    visited.append(temp_ticker)

ticker_list = new_ticker_list.copy()

In [181]:
ticker_data = yf.download(ticker_list, start=start_date, end=end_date, auto_adjust=True)
ticker_data

[*********************100%***********************]  36 of 36 completed

5 Failed downloads:
['CELG', 'ASDFA.TO', 'SQ', 'HDFC.NS']: YFTzMissingError('possibly delisted; no timezone found')
['XZO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-10-01 -> 2025-09-30) (Yahoo error = "Data doesn\'t exist for startDate = 1727755200, endDate = 1759204800")')


Price      Adj Close                            Close                     \
Ticker      ASDFA.TO CELG HDFC.NS  SQ XZO        ABBV AIM.TO        AMZN   
Date                                                                       
2024-10-01       NaN  NaN     NaN NaN NaN  188.798172   2.63  185.130005   
2024-10-02       NaN  NaN     NaN NaN NaN  188.663971   2.68  184.759995   
2024-10-03       NaN  NaN     NaN NaN NaN  187.350723   2.61  181.960007   
2024-10-04       NaN  NaN     NaN NaN NaN  186.238815   2.62  186.509995   
2024-10-05       NaN  NaN     NaN NaN NaN         NaN    NaN         NaN   
...              ...  ...     ...  ..  ..         ...    ...         ...   
2025-09-25       NaN  NaN     NaN NaN NaN  216.975800   3.15  218.149994   
2025-09-26       NaN  NaN     NaN NaN NaN  219.030991   3.18  219.779999   
2025-09-27       NaN  NaN     NaN NaN NaN         NaN    NaN         NaN   
2025-09-28       NaN  NaN     NaN NaN NaN         NaN    NaN         NaN   
2025-09-29       NaN  NaN     NaN NaN NaN  221.562729   3.12  222.169998   

Price                       ...      Volume                                    \
Ticker     ASDFA.TO   AUST  ...        SHOP         SLB         SO        SPG   
Date                        ...                                                 
2024-10-01      NaN  1.278  ...   5656200.0  14716500.0  4029900.0  1327100.0   
2024-10-02      NaN  1.270  ...   3110500.0  14614600.0  4869000.0  1321400.0   
2024-10-03      NaN  1.450  ...   4625400.0  13775400.0  6120700.0  1126700.0   
2024-10-04      NaN  1.258  ...   7737900.0  11294700.0  3432300.0  1211100.0   
2024-10-05      NaN    NaN  ...         NaN         NaN        NaN        NaN   
...             ...    ...  ...         ...         ...        ...        ...   
2025-09-25      NaN  1.790  ...   8511200.0  11619300.0  6968400.0  1303600.0   
2025-09-26      NaN  1.808  ...   7259500.0  14594600.0  5203400.0  1368100.0   
2025-09-27      NaN    NaN  ...         NaN         NaN        NaN        NaN   
2025-09-28      NaN    NaN  ...         NaN         NaN        NaN        NaN   
2025-09-29      NaN  1.790  ...  14702700.0  14983000.0  8002600.0  1282600.0   

Price                                                         
Ticker      SQ      SU.TO       T.TO         TD   VSP.TO XZO  
Date                                                          
2024-10-01 NaN  5659400.0  1938900.0  5242900.0  38400.0 NaN  
2024-10-02 NaN  3099000.0  3221700.0  1294300.0  21800.0 NaN  
2024-10-03 NaN  5150100.0  3113300.0  2775300.0  24100.0 NaN  
2024-10-04 NaN  3609400.0  2592100.0   993400.0  40700.0 NaN  
2024-10-05 NaN        NaN        NaN        NaN      NaN NaN  
...         ..        ...        ...        ...      ...  ..  
2025-09-25 NaN  8056200.0  2577600.0  1446500.0  36400.0 NaN  
2025-09-26 NaN  6431100.0  2884100.0  1555400.0  34000.0 NaN  
2025-09-27 NaN        NaN        NaN        NaN      NaN NaN  
2025-09-28 NaN        NaN        NaN        NaN      NaN NaN  
2025-09-29 NaN  5597500.0  3283900.0  4398400.0  23500.0 NaN  

[364 rows x 185 columns]

1. Remove all unwanted stocks and months:
    - Remove all foreign stocks, keep only the US and Canadian stocks
    - Remove all stocks with an average daily volume < 5,000 shares between Oct 1, 2024 and Sep 30, 2025 
    - Drop months for each stock with < 18 trading days


<font color='red'> Basically no new code here, but further explanation: by doing yf.download and then getting the new ticker_list, it automatically filters out any duplicate tickers, which was another thing that was in the ticker file
</font>

In [182]:
temp_list = ticker_data['Close']
ticker_list = temp_list.columns.tolist()
ticker_list

['ABBV',
 'AIM.TO',
 'AMZN',
 'ASDFA.TO',
 'AUST',
 'AW.TO',
 'AXP',
 'BK',
 'BTC-USD',
 'CELG',
 'CMCSA',
 'COST',
 'CSCO',
 'DUOL',
 'EXE.TO',
 'FTG.TO',
 'GC=F',
 'GCT',
 'GM',
 'GOOG',
 'HDFC.NS',
 'LOW',
 'ORCL',
 'PEP',
 'RY.TO',
 'SAP.TO',
 'SHOP',
 'SLB',
 'SO',
 'SPG',
 'SQ',
 'SU.TO',
 'T.TO',
 'TD',
 'VSP.TO',
 'XZO']

In [183]:
# Go through every month, remove any month for a ticker where their trading days are < 18
trading_days = ticker_data.index
filtered_ticker_data = ticker_data.copy()
for ticker in ticker_list:
    
    # gets the number of days in each month with values, basically meaning the number of trading days
    days_each_month = filtered_ticker_data['Close'][ticker].resample('ME').count()
    days_each_month.index = days_each_month.index.strftime('%Y-%m')

    # go through every day for this ticker, and set every day that's in a month with < 18 trading days to None
    for day in trading_days:
        #print(days_each_month.loc[day.strftime('%Y-%m')])
        if days_each_month.loc[day.strftime('%Y-%m')] < 18:
            # setting these values to None so that they are skipped when calculating numbers such as .mean() or .pct_change()
            filtered_ticker_data.loc[day, ('Close', ticker)] = None 
            filtered_ticker_data.loc[day, ('Volume', ticker)] = None


<font color='red'> No new code here, but further explanation: by filtering out the ticker_list for only tickers with >= 5000 average daily volume, tickers such as ASDFA.TO that were invalid will automatically get filtered out because they don't have any volume data
</font>

In [184]:
# Remove every stock with average trade volume < 5000 shares
volumes = filtered_ticker_data['Volume'].copy()
mean_volumes = volumes.mean()

min_volume = 5000

# The rest of the code will only reference stocks from filtered_tickers, so any stocks with volume < 5000 are removed
filtered_tickers = mean_volumes[mean_volumes >= min_volume].index.tolist()

# These are the remaining valid tickers
filtered_tickers

['ABBV',
 'AIM.TO',
 'AMZN',
 'AUST',
 'AW.TO',
 'AXP',
 'BK',
 'BTC-USD',
 'CMCSA',
 'COST',
 'CSCO',
 'DUOL',
 'EXE.TO',
 'FTG.TO',
 'GCT',
 'GM',
 'GOOG',
 'LOW',
 'ORCL',
 'PEP',
 'RY.TO',
 'SAP.TO',
 'SHOP',
 'SLB',
 'SO',
 'SPG',
 'SU.TO',
 'T.TO',
 'TD',
 'VSP.TO']

All the stocks that are leftover in this outputted list are the ones that have an average trading volume > 5000 during the time period

<font color='red'> NEW CODE: some tickers in the file were not stocks, such as BTC, so we have to check their quote type to see if they are equities. If they aren't, and are instead cryptocurrencies, for example, then we remove them.
</font>

In [185]:
# Remove all foreign stocks
fil = filtered_tickers.copy()

# Goes through all the tickers in the current list of tickers, gets their country, and if it's not US or Canada, it gets removed 
for ticker in fil:
    tick = yf.Ticker(ticker)
    info = tick.get_info()

    # NEW CODE in this section
    type_stock = info['quoteType']
    if type_stock != 'EQUITY':
        print(f'Ticker removed for not being a stock: {ticker} {type_stock}')
        filtered_tickers.remove(ticker)
    # End of new code
    
    else:
        country = info['country']
        #print(country)
        if country == 'United States' or country == 'Canada':
            continue
        
        print(f'Stock removed for not being a US or Canadian company: {ticker}')
        filtered_tickers.remove(ticker)

# These are the remaining valid tickers
filtered_tickers

Ticker removed for not being a stock: BTC-USD CRYPTOCURRENCY
Ticker removed for not being a stock: VSP.TO ETF


['ABBV',
 'AIM.TO',
 'AMZN',
 'AUST',
 'AW.TO',
 'AXP',
 'BK',
 'CMCSA',
 'COST',
 'CSCO',
 'DUOL',
 'EXE.TO',
 'FTG.TO',
 'GCT',
 'GM',
 'GOOG',
 'LOW',
 'ORCL',
 'PEP',
 'RY.TO',
 'SAP.TO',
 'SHOP',
 'SLB',
 'SO',
 'SPG',
 'SU.TO',
 'T.TO',
 'TD']

All the stocks that are leftover in this outputted list are US or Canadian companies 

2. After removing all the unwanted stocks, get data for each of the remaining stocks that will help us determine if they are stable and optimal for us to add to our portfolio:
    - Standard deviation
        - Tells us how volatile/risky the stock is, and we will look for stocks that have lower standard deviations (close to 0) to reduce the volatility and risk of our portfolio
    - Beta to the S&P 500
        - Tells us how closely the stock moves with the market. We want to look for stocks that have lower betas (close to 0 in absolute value) because it will mean that the stock's movements aren't very correlated with the market, and don't particularly go up or down when the market moves
    - Expected Monthly Returns
        - Tells us what percentage return we should expect to see per month. We want to look for stocks that have lower expected monthly returns (close to 0 in absolute value) because we want a portfolio that gives as close to 0% returns as possible

In [186]:
# Monthly Returns

close = filtered_ticker_data['Close'].copy()
close = close[filtered_tickers]
monthly_returns=close.resample('ME').first().pct_change()
monthly_returns.drop(index=monthly_returns.index[0], inplace=True)

# Expected Monthly Returns
monthly_returns_list = abs(monthly_returns.mean()) # do absolute value because we want it as close to 0 as possible
monthly_returns_list

Ticker
ABBV      0.011840
AIM.TO    0.019067
AMZN      0.020389
AUST      0.088910
AW.TO     0.006085
AXP       0.023071
BK        0.039552
CMCSA     0.012252
COST      0.008807
CSCO      0.026716
DUOL      0.018592
EXE.TO    0.036722
FTG.TO    0.064335
GCT       0.014757
GM        0.027519
GOOG      0.024478
LOW       0.001247
ORCL      0.035013
PEP       0.007982
RY.TO     0.020285
SAP.TO    0.017727
SHOP      0.061354
SLB       0.010342
SO        0.004910
SPG       0.010077
SU.TO     0.013006
T.TO      0.005994
TD        0.021169
dtype: float64

This output shows the expected monthly returns of each stock. Clearly, some of the stocks have way higher percent monthly returns than others. For example, C has a monthly return of 4%, which is almost 100x more than the returns of QCOM, at only 0.05%. So, later on, when we rank the stocks, QCOM will be ranked higher than C because of its close-to-0 returns.

In [187]:
# Beta

# Get the prices of the S&P 500, and then we will compare its movements vs. all the other stocks
market_ticker = yf.Ticker('^GSPC')
market_hist = market_ticker.history(start=start_date, end=end_date)
market_close = market_hist['Close']
market_close.index = market_close.index.tz_localize(None) # have to remove the timezones or else the dataframes can't be merged
#market_close.index = market_close.index.strftime('%Y-%m-%d')

# Putting the price data for all the other stocks together with the S&P 500
close_beta = close.copy()
close_beta.index = close_beta.index.tz_localize(None)
#close_beta.index = close_beta.index.strftime('%Y-%m-%d')
close_beta = pd.merge(close_beta, market_close, left_index=True, right_index=True, how='inner')

# Getting the beta between the market and all the stocks
close_beta_returns = close_beta.resample('ME').ffill().pct_change()
close_beta_returns = close_beta_returns.iloc[1:]
market_variance = close_beta_returns['Close'].var()

# do absolute value because we want it as close to 0 as possible from either direction
beta = (abs(close_beta_returns.cov()/market_variance))['Close']
beta = beta.drop('Close')
beta

ABBV      0.437586
AIM.TO    0.133327
AMZN      1.628868
AUST      0.659081
AW.TO     0.670606
AXP       1.810766
BK        1.366941
CMCSA     0.085609
COST      0.834254
CSCO      0.982807
DUOL      1.398500
EXE.TO    0.043799
FTG.TO    0.937975
GCT       4.497879
GM        1.068520
GOOG      1.030516
LOW       0.942976
ORCL      3.216639
PEP       0.238420
RY.TO     0.831371
SAP.TO    0.025142
SHOP      3.416300
SLB       0.958139
SO        0.052204
SPG       1.067863
SU.TO     0.765645
T.TO      0.624765
TD        0.602407
Name: Close, dtype: float64

This output shows the beta of each stock vs. the S&P 500. Clearly, some of the stocks have way higher betas than others. For example, AMZN has a beta of 1.63, which means that it moves more dramatically than the market. If the market changed by 10%, then AMZN would change by 16.3%, which is pretty sensitive. Compared to MRK, which has a beta of 0.052, MRK usually moves less dramatically than the market, making it not very sensitive to the market and therefore a more stable stock. So, later on, when we rank the stocks, MRK will be ranked higher than AMZN because of its low beta.

In [188]:
# Standard Deviations
standard_deviation = monthly_returns.std()
standard_deviation

Ticker
ABBV      0.065391
AIM.TO    0.059915
AMZN      0.070866
AUST      0.459296
AW.TO     0.050237
AXP       0.079633
BK        0.059541
CMCSA     0.076527
COST      0.069411
CSCO      0.049676
DUOL      0.186372
EXE.TO    0.100043
FTG.TO    0.088269
GCT       0.181761
GM        0.072780
GOOG      0.079548
LOW       0.063216
ORCL      0.125184
PEP       0.054107
RY.TO     0.044220
SAP.TO    0.076651
SHOP      0.144005
SLB       0.094263
SO        0.039638
SPG       0.063670
SU.TO     0.058194
T.TO      0.056125
TD        0.066615
dtype: float64

This output shows the standard deviation of each stock. Clearly, some of the stocks have way higher standard deviations than others. For example, AXTI has a standard deviation of 0.25, which is much higher than BLK, with a standard deviation of 0.04. This means that BLK is less risky/volatile than AXTI, making it a better candidate for the portfolio. So, later on, when we rank the stocks, BLK will be ranked higher than AXTI because of its low standard deviation

Each stock has been ranked on the 3 metrics now, so we have to find a way to combine those rankings in the 3 metrics into one metric. We did this by normalizing each of the rankings in the 3 metrics, meaning the stock ranked the lowest in a metric will get a score of 0, and the stock ranked the highest will get a score of 1, and all the other stocks are between 0 and 1. Importantly, this score keeps the magnitude of the stock in each of the metrics: if two stocks both had very high monthly returns, their scores might be 0.999 and 1 rather than a flat increment. So, if one stock has significantly lower returns than every other stock, it will be rewarded proportionally. The 0-1 scores for each metric are added together to get one final score between 0 and 3, and the stocks will be sorted in ascending order based on their final score. If a stock has low returns, low beta, and low standard deviation, then it is likely that it will be at the front of the list (good score)

In [189]:
# Convert the lists we have of monthly returns, beta, and standard deviation into a score that we can rank the stocks by

# For each of the metrics we measured, we'll convert the list into a score between 0 and 1, where 0 is the safest/lowest return etc.
# and 1 is the most volatile/highest return in absolute value etc.
# At the end, the scores are combined together, and whichever stocks have the lowest scores will be the most favoured

# Normalizes the monthly returns to a range between 0 and 1, where the lowest value becomes 0 and the largest value becomes 1
monthly_min = monthly_returns_list.min()
monthly_max = monthly_returns_list.max()
monthly_score = (monthly_returns_list - monthly_min) / (monthly_max - monthly_min)

# Now each stock has been given a rating between 0 and 1 (closer to 0 is better)
monthly_score


Ticker
ABBV      0.120838
AIM.TO    0.203288
AMZN      0.218363
AUST      1.000000
AW.TO     0.055199
AXP       0.248955
BK        0.436961
CMCSA     0.125537
COST      0.086244
CSCO      0.290533
DUOL      0.197867
EXE.TO    0.404681
FTG.TO    0.719666
GCT       0.154116
GM        0.299699
GOOG      0.265011
LOW       0.000000
ORCL      0.385187
PEP       0.076835
RY.TO     0.217181
SAP.TO    0.187998
SHOP      0.685656
SLB       0.103751
SO        0.041792
SPG       0.100735
SU.TO     0.134144
T.TO      0.054152
TD        0.227264
dtype: float64

In [190]:
# Normalizes the betas to a range between 0 and 1, where the lowest value becomes 0 and the largest value becomes 1
beta_min = beta.min()
beta_max = beta.max()
beta_score = (beta - beta_min) / (beta_max - beta_min)

# Now each stock has been given a rating between 0 and 1 (closer to 0 is better)
beta_score

ABBV      0.092213
AIM.TO    0.024188
AMZN      0.358556
AUST      0.141734
AW.TO     0.144311
AXP       0.399224
BK        0.299995
CMCSA     0.013519
COST      0.180899
CSCO      0.214112
DUOL      0.307051
EXE.TO    0.004171
FTG.TO    0.204088
GCT       1.000000
GM        0.233275
GOOG      0.224778
LOW       0.205206
ORCL      0.713545
PEP       0.047684
RY.TO     0.180254
SAP.TO    0.000000
SHOP      0.758184
SLB       0.208596
SO        0.006050
SPG       0.233128
SU.TO     0.165559
T.TO      0.134062
TD        0.129063
Name: Close, dtype: float64

In [191]:
# Normalizes the standard deviation to a range between 0 and 1, where the lowest value becomes 0 and the largest value becomes 1
standard_deviation_min = standard_deviation.min()
standard_deviation_max = standard_deviation.max()
standard_deviation_score = (standard_deviation - standard_deviation_min) / (standard_deviation_max - standard_deviation_min)

# Now each stock has been given a rating between 0 and 1 (closer to 0 is better)
standard_deviation_score

Ticker
ABBV      0.061365
AIM.TO    0.048318
AMZN      0.074411
AUST      1.000000
AW.TO     0.025256
AXP       0.095304
BK        0.047426
CMCSA     0.087903
COST      0.070945
CSCO      0.023920
DUOL      0.349651
EXE.TO    0.143938
FTG.TO    0.115883
GCT       0.338663
GM        0.078973
GOOG      0.095101
LOW       0.056183
ORCL      0.203846
PEP       0.034478
RY.TO     0.010917
SAP.TO    0.088198
SHOP      0.248695
SLB       0.130165
SO        0.000000
SPG       0.057265
SU.TO     0.044216
T.TO      0.039286
TD        0.064283
dtype: float64

3. Now that we have data for all the stocks, combine the data into a score and rank the stocks on the 3 metrics, getting a sorted list in order of which stocks are the best to invest in

In [192]:
# Weigh each score equally, add them together (now it's a score between 0 and 3)
overall_score = monthly_score + beta_score + standard_deviation_score
overall_temp = overall_score.copy()

# Sort the stocks based on their scores, the lower the better, so those get pushed to the front 
# and the higher scores get pushed to the back
overall_score = overall_score.sort_values()
overall_score

Ticker
SO        0.047842
PEP       0.158997
AW.TO     0.224765
CMCSA     0.226959
T.TO      0.227499
LOW       0.261389
ABBV      0.274416
AIM.TO    0.275794
SAP.TO    0.276197
COST      0.338087
SU.TO     0.343919
SPG       0.391128
RY.TO     0.408352
TD        0.420609
SLB       0.442512
CSCO      0.528564
EXE.TO    0.552791
GOOG      0.584891
GM        0.611946
AMZN      0.651330
AXP       0.743483
BK        0.784382
DUOL      0.854569
FTG.TO    1.039638
ORCL      1.302578
GCT       1.492779
SHOP      1.692535
AUST      2.141734
dtype: float64

In [193]:
# Only get the tickers, the best scoring tickers are in the front, worst scoring are in the back
filtered_tickers = overall_score.index.tolist()
filtered_tickers

['SO',
 'PEP',
 'AW.TO',
 'CMCSA',
 'T.TO',
 'LOW',
 'ABBV',
 'AIM.TO',
 'SAP.TO',
 'COST',
 'SU.TO',
 'SPG',
 'RY.TO',
 'TD',
 'SLB',
 'CSCO',
 'EXE.TO',
 'GOOG',
 'GM',
 'AMZN',
 'AXP',
 'BK',
 'DUOL',
 'FTG.TO',
 'ORCL',
 'GCT',
 'SHOP',
 'AUST']

In [194]:

# Putting the scores from the 3 metrics and the overall score into a dataframe
stocks_graph = standard_deviation_score.index
monthly_graph = monthly_score.values
beta_graph = beta_score.values
std_graph = standard_deviation_score.values
overall_graph = overall_temp.values
graph_df = {
    'Expected Monthly Returns': monthly_graph,
    'Beta': beta_graph,
    'Standard Deviation': std_graph,
    'Overall': overall_graph
}
graph_df = pd.DataFrame(graph_df)
graph_df.index = stocks_graph
graph_df

,Expected Monthly Returns,Beta,Standard Deviation,Overall
Ticker,,,,
ABBV,0.120838,0.092213,0.061365,0.274416
AIM.TO,0.203288,0.024188,0.048318,0.275794
AMZN,0.218363,0.358556,0.074411,0.651330
AUST,1.000000,0.141734,1.000000,2.141734
AW.TO,0.055199,0.144311,0.025256,0.224765
AXP,0.248955,0.399224,0.095304,0.743483
BK,0.436961,0.299995,0.047426,0.784382
CMCSA,0.125537,0.013519,0.087903,0.226959
COST,0.086244,0.180899,0.070945,0.338087


After all the scores were calculated, we can see that most of the time, stocks that get a low (good) score in one metric tend to get low scores in all metrics. For example, the best scoring stock, PFE, had scores < 0.1 for all 3 metrics, while the worst scoring stock, AXTI, had scores > 0.75 for all 3 metrics. For SHOP.TO, it got the worst score (1) for both expected monthly returns and beta, but only had a 0.49 for standard deviation. This means that although Shopify has extremely high returns and moves more extremely than the market, its volatility/risk is not as high as some of the other stocks, making it still rank second last, but not the worst stock to invest in.

4. Based on the ranking, add stocks to the portfolio one at a time 
    - We want 25 stocks (maximum amount allowed) at the end, with each being weighed equally because we want to diversify as much as possible and not get hurt by company-based scandals or industry-wide down/upturns. 
    - First, add the large and small-cap stocks that rank the highest in the scores
    - We don't just choose the top 25 stocks based on the ranking because that ranking doesn't take into account how correlated each stock is to one another
    - For example, if the top 25 stocks are all in the same industry, then they will probably be highly correlated with each other, making our portfolio not very diversified
    - This is why we will add the stocks one at a time and making sure that each additional stock is not too correlated with the stocks already in the portfolio
    - For each of the next stocks (going through the stocks based on their rankings):
        - If there are already 10 (0.4 x 25) stocks in the portfolio that are in the same industry, then skip
        - Get the average correlation between them and each of the existing stocks already in the portfolio
        - If it’s below a certain threshold, then add it to the portfolio
        - For the first few stocks, we will make the threshold higher/more lenient since there are fewer stocks to compare it to
        - Later on, if we have already chosen a few of our stocks, we will reduce the threshold, since if the average correlation for a stock is still high, then it means that it's strongly correlated with many of the stocks already in the portfolio, which is not optimal
        - If there are already 25 stocks, then break out the loop/stop
        - Change the correlation thresholds and run through the loop again, adding one stock at a time to an empty portfolio

    - After every iteration, we will have a portfolio of stocks. (If the portfolio has fewer than 25 stocks, skip this part)
    - We will find the expected daily return of that portfolio, and if the absolute value of the return is lower than that of our current best portfolio (closer to 0), then this becomes the new best portfolio
    - After all possible correlation thresholds are gone through, we will have the most optimal portfolio

In [195]:
# Before we add any stocks, we have to get information on every stock's sector, market cap, price, currency
# Sectors will be used to count if the portfolio has gone over 40% in a single sector
# Currency is used to convert USD to CAD
# Market Cap is used to get a stock that has a big market cap and another that has a small market cap
# Price will be used to calculate how many shares to buy at the end

# convert all US prices to CAD
exchange_rate = yf.Ticker("USDCAD=X").fast_info["last_price"]

# get_info for all stocks
info_stocks = {
    'Ticker': [],
    'Sector': [],
    'MarketCap': [],
    'Price': [],
    'Currency': []
}
sectors_set = set() # list of all the unique sectors

# Go through all the tickers, add the needed data to the data frame
for ticker in filtered_tickers:
    tick = yf.Ticker(ticker)
    info = tick.get_info()
    info_stocks['Ticker'].append(ticker)
    sector = info['sector']
    info_stocks['Sector'].append(sector)
    currency = info['currency']
    info_stocks['Currency'].append(currency)
    price = tick.fast_info['previousClose']

    # adds all the sectors of all the stocks to this set, and it only keeps unique ones 
    sectors_set.add(sector)
      
    market_cap = info['marketCap']

    # if the currency is in USD, then convert to CAD
    if currency == 'USD':
        info_stocks['MarketCap'].append(market_cap * exchange_rate)
        info_stocks['Price'].append(price* exchange_rate)
    else:
        info_stocks['MarketCap'].append(market_cap)
        info_stocks['Price'].append(price)

info_stocks = pd.DataFrame(info_stocks).set_index('Ticker')
sectors = list(sectors_set)
print(f'The sectors in this list of stocks are: {sectors}')
info_stocks

The sectors in this list of stocks are: ['Basic Materials', 'Communication Services', 'Consumer Defensive', 'Energy', 'Real Estate', 'Technology', 'Financial Services', 'Healthcare', 'Utilities', 'Industrials', 'Consumer Cyclical']


,Sector,MarketCap,Price,Currency
Ticker,,,,
SO,Utilities,1.407266e+11,126.234004,USD
PEP,Consumer Defensive,2.856206e+11,208.230003,USD
AW.TO,Consumer Cyclical,8.977570e+08,37.380001,CAD
CMCSA,Communication Services,1.382192e+11,37.267083,USD
T.TO,Communication Services,2.839032e+10,18.330000,CAD
LOW,Consumer Cyclical,1.907810e+11,339.625576,USD
ABBV,Healthcare,5.644532e+11,319.333320,USD
AIM.TO,Basic Materials,2.519520e+08,2.790000,CAD
SAP.TO,Consumer Defensive,1.624666e+10,39.060000,CAD


This data frame shows all the needed data to calculate what stocks will be put in the portfolio, and the sectors list shows all the sectors from the stocks, so later on we can count how many stocks there are in each sector as we add stocks to the portfolio and make sure it doesn't go over the 40% (10 stocks / 25) limit

In [196]:
# stocks_portfolio will be the list of 25 stocks that we will purchase at the end
stocks_portfolio = []
lst = filtered_tickers.copy()
# Add the large market cap stock with the highest score
for ticker in lst:
    market_cap = info_stocks.loc[ticker, 'MarketCap']
    # if it's a large market cap stock
    if market_cap > 10000000000:
            stocks_portfolio.append(ticker) # add to the 25 stocks we will purchase
            filtered_tickers.remove(ticker) # remove from the current candidates pool
            break

# Add the small market cap stock with the highest score
for ticker in lst:
    market_cap = info_stocks.loc[ticker, 'MarketCap']
    if market_cap < 2000000000:
            stocks_portfolio.append(ticker) # add to the 25 stocks we will purchase
            filtered_tickers.remove(ticker) # remove from the current candidates pool
            break

print(f'The best-rated large market cap stock and small market cap stock: {stocks_portfolio}')



The best-rated large market cap stock and small market cap stock: ['SO', 'AW.TO']


Next part goes over how each stock is added to the final portfolio:

    - For each of the next stocks (going through the stocks based on their rankings):
        - If there are already 10 (0.4 x 25) stocks in the portfolio that are in the same industry, then skip
        - Get the average correlation between them and each of the existing stocks already in the portfolio
        - If it’s below a certain threshold, then add it to the portfolio
        - For the first few stocks, we will make the threshold higher/more lenient since there are fewer stocks to compare it to
        - Later on, if we have already chosen a few of our stocks, we will reduce the threshold, since if the average correlation for a stock is still high, then it means that it's strongly correlated with many of the stocks already in the portfolio, which is not optimal
        - If there are already 25 stocks, then break out the loop/stop
        - Change the correlation thresholds and run through the loop again, adding one stock at a time to an empty portfolio
    - After every iteration, we will have a portfolio of stocks. (If the portfolio has fewer than 25 stocks, skip this part)
    - We will find the expected daily return of that portfolio, and if the absolute value of the return is lower than that of our current best portfolio (closer to 0), then this becomes the new best portfolio
    - After all possible correlation thresholds are gone through, we will have the most optimal portfolio

In [197]:
# default is 25 stocks, but if the length of valid stocks are < 25, then just use that entire length
max_stocks = 25
if len(lst) < max_stocks:
    max_stocks = len(lst)

adj_prices = filtered_ticker_data['Close']
# Function that adds one stock at a time depending on if the stock is too correlated with the portfolio or not
# takes in 3 correlation threshold values, corr1 is for when there are <= 1/3 of the 25 stocks added to the portfolio,
# corr2 is for when there are <= 2/3 of the 25 stocks added to the portfolio, and corr3 is for the final 1/3 of stocks to be added
# further explanation down below
def add_stocks(corr1, corr2, corr3):
    # the list that will be outputted from this function
    temp_stocks_portfolio = stocks_portfolio.copy()

    # keeps track of all the sectors that have been encountered as a frequency list, and if the count hits 40% of the max_stocks,
    # the for loop won't accept anymore stocks from that industry
    sector_count = [0] * len(sectors)
    
    # go through all the stocks in order from best ranked to worst ranked
    for i, ticker in enumerate(filtered_tickers):

        # if there are already 25 stocks in the portfolio, exit out with the current portfolio
        if len(temp_stocks_portfolio) == max_stocks:
            return temp_stocks_portfolio

        # find the sector of the current stock, if that sector already has enough stocks, then don't add it
        sector = info_stocks.loc[ticker, 'Sector']
        if sector_count[sectors.index(sector)] == 0.4*max_stocks:
            continue


        curr_adj_price = adj_prices[ticker]
        corr_sum = 0

        # find the average correlation between the current stock and all the stocks already in the portfolio
        for tick in temp_stocks_portfolio:
            corr_sum += curr_adj_price.corr(adj_prices[tick])           
        corr_sum /= len(temp_stocks_portfolio)

        # if there aren't many stocks already in the portfolio, then it checks the average correlation against corr1
        if len(temp_stocks_portfolio) < max_stocks* (1/3):
            # if the average correlation is below the threshold, then it is good enough to be added
            if corr_sum < corr1:
                temp_stocks_portfolio.append(ticker) # add stock to portfolio
                sector_count[sectors.index(sector)] += 1 # add 1 to the sector that this stock was in

        # if there are a few stocks already in the portfolio, then it checks the average correlation against corr2
        elif len(temp_stocks_portfolio) < max_stocks * (2/3):
            # if the average correlation is below the threshold, then it is good enough to be added
            if corr_sum < corr2:
                temp_stocks_portfolio.append(ticker) # add stock to portfolio
                sector_count[sectors.index(sector)] += 1 # add 1 to the sector that this stock was in
                
        # if there are a lot stocks already in the portfolio, then it checks the average correlation against corr3
        else:
            # if the average correlation is below the threshold, then it is good enough to be added
            if corr_sum < corr3:
                temp_stocks_portfolio.append(ticker) # add stock to portfolio
                sector_count[sectors.index(sector)] += 1 # add 1 to the sector that this stock was in
    return temp_stocks_portfolio


In [ ]:
# takes the function from before, runs it for all possible combinations of correlation thresholds between 0 and 1
# for every successful portfolio made from one run of the function, it will check its daily returns
# if the absolute value of its daily returns is less than the previous best portfolio's daily returns, then this current one 
# becomes the best

min_returns = 100
final_portfolio = []
optimal_corr_thres = []
# goes through every correlation threshold from 0 to 1 in increments of 0.1
for i in range(11): # 11 is not included
    for j in range(11):
        for k in range(11):
            # gets the portfolio from the function above
            output = add_stocks(i*0.1, j*0.1, k*0.1)

            # if there aren't 25 stocks in the portfolio, then try again with higher correlation thresholds
            if len(output) < max_stocks:
                continue

            # finding the expected daily returns
            portfolio_prices = adj_prices[output]
            daily_returns = portfolio_prices.pct_change(fill_method=None)
            daily_returns.drop(index=daily_returns.index[0], inplace=True)
            
            portfolio_daily_returns = daily_returns.mean(axis=1)
            expected_portfolio_returns = portfolio_daily_returns.mean()

            # if this return is < the previous best minimum return, then this portfolio becomes the new minimum/best
            if abs(expected_portfolio_returns) < abs(min_returns):
                final_portfolio = output
                min_returns = expected_portfolio_returns
                print(f'Current lowest daily returns by a portfolio: {min_returns}')
                print(f'Current portfolio that had the lowest daily returns: {final_portfolio}')
                optimal_corr_thres = [i*0.1, j*0.1, k*0.1]
                print(f'Current optimal correlation thresholds: {optimal_corr_thres}')
                print()
print()
print(f'Lowest daily returns by a portfolio: {min_returns}')
print(f'Portfolio that had the lowest daily returns: {final_portfolio}')
print(f'Optimal correlation thresholds: {optimal_corr_thres}')

Current lowest daily returns by a portfolio: 0.001059701090263812
Current portfolio that had the lowest daily returns: ['SO', 'AW.TO', 'PEP', 'CMCSA', 'T.TO', 'LOW', 'ABBV', 'AIM.TO', 'COST', 'SU.TO', 'SPG', 'RY.TO', 'TD', 'SLB', 'CSCO', 'EXE.TO', 'GOOG', 'GM', 'AMZN', 'AXP', 'BK', 'DUOL', 'FTG.TO', 'ORCL', 'GCT']
Current optimal correlation thresholds: [0.30000000000000004, 0.4, 0.5]

Current lowest daily returns by a portfolio: 0.0010583219720341906
Current portfolio that had the lowest daily returns: ['SO', 'AW.TO', 'PEP', 'CMCSA', 'T.TO', 'LOW', 'ABBV', 'AIM.TO', 'SAP.TO', 'COST', 'SU.TO', 'SPG', 'TD', 'SLB', 'CSCO', 'EXE.TO', 'GOOG', 'GM', 'AMZN', 'AXP', 'BK', 'DUOL', 'FTG.TO', 'ORCL', 'GCT']
Current optimal correlation thresholds: [0.5, 0.4, 0.5]

Current lowest daily returns by a portfolio: 0.000954688558220071
Current portfolio that had the lowest daily returns: ['SO', 'AW.TO', 'PEP', 'CMCSA', 'T.TO', 'LOW', 'ABBV', 'AIM.TO', 'SAP.TO', 'COST', 'SU.TO', 'SPG', 'RY.TO', 'TD', 'SL

As the code runs more combinations of correlation thresholds, the minimum expected daily returns keep going down until they reach 0.03%. At the beginning, the best minimum expected daily returns were 0.05%, but after increasing some of the threshold values, it reached 0.03%. It seems like for the 25 stocks, the most optimal combination ended being the top 24 stocks that were originally ranked plus the required small market cap stock. This validates the rankings that were done before, as they individually had the lowest returns/beta/standard deviations, and their combined portfolio also had the lowest returns. It also shows that these 25 stocks are pretty diversified, as all the stocks were able to fit under the 0.3 correlation threshold, so none of the stocks were highly correlated with each other. Other portfolios either had correlation thresholds that were too tight, leading to all the good stocks with low returns, beta, and volatility not having low enough average correlation to fit in the portfolio, or had correlation thresholds that were too loose, leading to too many stocks going into the portfolio that were highly correlated with each other, leading to high daily returns due to the lack of diversification. 

5. Purchase the 25 stocks in the portfolio
    - Give equal (4%) weighting to each stock
    - Deduct fees for purchasing, either 2.15 USD flat or 0.001 USD per share purchased (whichever is smaller)

In [ ]:
# create final data frame
info_stocks = info_stocks.loc[final_portfolio]

info_stocks = info_stocks.drop('Sector', axis=1)
info_stocks = info_stocks.drop('MarketCap', axis=1)
info_stocks.reset_index(inplace=True)



In [ ]:
# purchase the stocks
total_money = 1000000
# convert USD fees to CAD
fee = 0.001 * exchange_rate
max_fee = 2.15 * exchange_rate

curr_fee = 0
shares_temp = []

# equal weights for each stock
weight = 1/len(info_stocks.index)
weights = []

# get the number of shares for each stock if we used the $0.001 USD fee per share
for i in range(len(info_stocks.index)):
    weights.append(weight)
    price = info_stocks.loc[i, 'Price']
    shares = (total_money*weight) / (price + fee) 
    shares_temp.append(shares)
    # all shares need to pay the same fee
    curr_fee += shares * fee

# if the $0.001 USD fee per share result led to a higher total fee than $2.15 USD, then just use $2.15 total instead
if max_fee < curr_fee:
    shares_temp = []

    # for the $2.15 total fee, subtract the fee from the total $1 million 
    # and find how many shares can be bought from the remaining money
    total_money -= max_fee
    for i in range(len(info_stocks.index)):
        price = info_stocks.loc[i, 'Price']
        shares = (total_money*weight) / price
        shares_temp.append(shares)

info_stocks['Shares'] = shares_temp
info_stocks['Value'] = info_stocks['Shares'] * info_stocks['Price']
info_stocks['Weight'] = weights
Portfolio_Final = info_stocks
Portfolio_Final

In [ ]:
# add all the values together
total_portfolio_value = 0
total_weight = 0
for i in range(len(info_stocks.index)):
    total_portfolio_value += info_stocks.loc[i, 'Value']
    total_weight += info_stocks.loc[i, 'Weight']
print(f'Total portfolio value minus fees is: ${total_portfolio_value:.2f}')
print(f'Total portfolio weight is: {total_weight * 100:.0f}%')

Makes sense for it to be 999996.97 CAD because that's the same as 1 million CAD - 2.15 USD

In [ ]:
# We don't know how to find our group number
Portfolio_Final[['Ticker', 'Shares']].to_csv('Stocks_Group_XX.csv')

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Ole, Rudra, Sheraz